# Récupérer des images depuis swisstopo

Le but de ce notebook est de tester la possibilité d'utiliser l'API STAC (SpatioTemporal Asset Catalog) afin de lister et télécharger des images d'intérêt depuis swisstopo.

In [1]:
import os
import requests
import json

## Récupération du nom du jeu de données SWISSIMAGES via l'API

Il faut récupérer le nom du jeu de données [SWISSMAGES](https://www.swisstopo.admin.ch/fr/orthophotos-swissimage-10-cm). Les [infos générales](https://www.geo.admin.ch/fr/interface-rest-api-stac/) de l'API indiquent que l'adresse principale est `https://data.geo.admin.ch/api/stac/v0.9/` et la documentation détaillée est disponible [ici](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html).

In [161]:
stac_api_url = ("https://data.geo.admin.ch/api/stac/v0.9")

On récupère les ids de toutes les collections disponibles ([doc](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html#tag/Data/operation/getCollections)). Attention, les collections sont limités à 100 dans l'array `collections` retourné. Il faut utiliser le lien avec la `rel : "next"` contenu dans l'array `links` de l'objet contenant `{collections : [...], links : [...]}` !

In [162]:
# Stockage des ids
collection_ids = []

# GET sur les collections
response = requests.get(f"{stac_api_url}/collections")

# Vérification du succès
if response.status_code == 200:
    # Parse le json
    collections_data = response.json()

    # Extraire les ids des collections
    collection_ids.extend(collection["id"] for collection in collections_data.get("collections"))

    # Enchaîner avec les prochaines pages/liens
    while True:
        # Récupère le lien avec next ou retourne None si absent (dernier lien)
        next_link = next((link for link in collections_data.get("links") if link.get("rel") == "next"), None)
        if next_link:
            # Récupérer l'url
            next_url = next_link["href"]

            # Répéter les opérations précédentes
            response = requests.get(next_url)
            
            # Vérification du succès
            if response.status_code == 200:
                # Parse le json
                collections_data = response.json()

                # Extraire les ids des collections
                collection_ids.extend(collection["id"] for collection in collections_data.get("collections"))
            else:
                print("Échec. Erreur", response.status_code)
                break
        else:
            break  # Plus de lien "next"
else:
    print("Échec. Erreur", response.status_code)

print('\n'.join(collection_ids[0:3]))

ch.agroscope.feuchtflaechenpotential-kulturlandschaft
ch.are.agglomerationsverkehr
ch.are.alpenkonvention


On identifie les fournisseurs de données, qui viennent juste après le `ch.`. Ce sont généralement les abréviations allemandes des offices fédéraux (are (ODT), astra (OFROU), etc.).

In [163]:
fournisseurs = set(id.split(".")[1] for id in collection_ids)
print(sorted(fournisseurs))

['agroscope', 'are', 'armasuisse', 'astra', 'babs', 'bafu', 'bag', 'bak', 'bakom', 'baspo', 'bav', 'bazl', 'bfe', 'bfs', 'blw', 'ensi', 'meteoschweiz', 'pronatura', 'sem', 'swisstopo', 'swisstopo-vd', 'vbs']


Le founisseur qui nous intéresse est `swisstopo`, on regarde donc les jeux de données mis à disposition qui contiennent le mot `image`

In [164]:
for collection_id in collection_ids:
    if "swisstopo" in collection_id and "image" in collection_id:
        print(collection_id)

ch.swisstopo.swissimage-dop10


Le jeu de données des images est donc `ch.swisstopo.swissimage-dop10`

## Jeu de données SWISSIMAGE - Infos et téléchargement(s)

In [165]:
id_swimage = "ch.swisstopo.swissimage-dop10"

In [166]:
# GET sur le jeu de données
response = requests.get(f"{stac_api_url}/collections/{id_swimage}")
infos_jdd = response.json()
# La description en anglais du jdd
infos_jdd.get("description")

'The orthophoto mosaic SWISSIMAGE 10 cm is a composition of new digital color aerial photographs over the whole of Switzerland with a ground resolution of 10 cm in the plain areas and main alpine valleys and 25 cm over the Alps. It is updated in a cycle of 3 years since 2018.'

In [167]:
# Système de projection et résolution des images
infos_jdd.get("summaries")

{'proj:epsg': [2056], 'eo:gsd': [0.1, 2.0]}

Il est possible de restreindre la recherche à l'aide de la bbox de la zone.
Par exemple, on pourrait donner la suivante (format xmin, ymin, xmax, ymax) `bbox=7.34791,46.23141,7.36087,46.24042` afin de trouver toutes les images correspondant à cette zone.

In [168]:
ymin, xmin, ymax, xmax = 46.23141, 7.34791, 46.24042, 7.36087
bbox = [xmin, ymin, xmax, ymax]
response = requests.get(f"{stac_api_url}/collections/{id_swimage}/items?bbox={", ".join(str(coord) for coord in bbox)}")

In [169]:
features = response.json().get("features")

46.23141, 7.34791, 46.24042, 7.36087 || 2593-1120

In [170]:
assets_links = []

for feature in features:
    for name, properties in feature.get("assets").items():
        if properties.get('eo:gsd') == 0.1:
            assets_links.append(properties.get('href'))
assets_links

['https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2017_2593-1119/swissimage-dop10_2017_2593-1119_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2017_2593-1120/swissimage-dop10_2017_2593-1120_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2017_2593-1121/swissimage-dop10_2017_2593-1121_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2017_2594-1120/swissimage-dop10_2017_2594-1120_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2017_2594-1121/swissimage-dop10_2017_2594-1121_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2593-1119/swissimage-dop10_2020_2593-1119_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2593-1120/swissimage-dop10_2020_2593-1120_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.

In [171]:
image_years = {}

for url in assets_links:
    # Extraire l'id de l'image "XXXX-XXXX"
    image_id = url.split('/')[-2].split('_')[2]

    # Extraire l'année de l'image 
    year = int(url.split('_')[1])

    # Ajouter l'année max correspondant à l'image
    if image_id in image_years:
        image_years[image_id] = max(image_years[image_id], year)
    else:
        image_years[image_id] = year

# Garder uniquement les urls avec l'image la plus récente
most_recent_assets_links = [
    url for url in assets_links if int(url.split('/')[-2].split('_')[1]) == image_years[url.split('/')[-2].split('_')[2]]
]
most_recent_assets_links

['https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2593-1119/swissimage-dop10_2020_2593-1119_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2593-1120/swissimage-dop10_2020_2593-1120_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2593-1121/swissimage-dop10_2020_2593-1121_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2594-1120/swissimage-dop10_2020_2594-1120_0.1_2056.tif',
 'https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2020_2594-1121/swissimage-dop10_2020_2594-1121_0.1_2056.tif']

#### Télécharger les images

In [173]:
for url in most_recent_assets_links:
    filename = url.split('/')[-1]
    filename = filename.split("_")[2] + ".tif"
    filepath = os.path.join("../data", "download_test", filename)
    response = requests.get(url)
    with open(filepath, 'wb') as f:
        f.write(response.content)

## Téléchargement à l'aide des bbox

Regrouper les étapes précédentes dans une fonction

In [17]:
def download_images(xmin, ymin, xmax, ymax, output_dir):
    # URL STAC
    stac_api_url = ("https://data.geo.admin.ch/api/stac/v0.9")
    # ID jdd
    id_swimage = "ch.swisstopo.swissimage-dop10"
    
    # Regrouper en bbox array
    bbox = [xmin, ymin, xmax, ymax]

    # Faire la requete
    response = requests.get(f"{stac_api_url}/collections/{id_swimage}/items?bbox={','.join(str(coord) for coord in bbox)}")
    features = response.json().get('features', [])

    # Extraire les urls
    assets_links = []
    for feature in features:
        for name, properties in feature.get("assets", {}).items():
            if properties.get('eo:gsd') == 0.1:
                assets_links.append(properties.get('href'))

    # Trouver les images les plus récentes
    image_years = {}
    for url in assets_links:
        image_id = url.split('/')[-2].split('_')[2]
        year = int(url.split('_')[1])
        if image_id in image_years:
            image_years[image_id] = max(image_years[image_id], year)
        else:
            image_years[image_id] = year

    # Liste des urls des images les plus récentes
    most_recent_assets_links = [
        url for url in assets_links if int(url.split('/')[-2].split('_')[1]) == image_years[url.split('/')[-2].split('_')[2]]
    ]

    nb_images = len(most_recent_assets_links)

    # S'assurer que la sortie existe
    os.makedirs(output_dir, exist_ok=True)

    # Télécharger les images
    for url in most_recent_assets_links:
        filename = url.split('/')[-1]
        filename = filename.split("_")[2] + ".tif"
        filepath = os.path.join(output_dir, filename)
        if os.path.exists(filepath):
            nb_images -= 1
            continue
        response = requests.get(url)
        with open(filepath, 'wb') as f:
            f.write(response.content)
    
    return nb_images 

In [15]:
with open("../data/bounding_boxes.json", 'r', encoding="utf-8") as file:
    bounding_boxes = json.load(file)

In [22]:
for bbox in bounding_boxes:
    nb_imgs = download_images(bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"], "../data/download_by_bbox")
    print(f"Downloaded {nb_imgs} images.")

Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 images.
Downloaded 0 